### Feature tools

It is automated feature tools Transforms Transactional and relational datasets into feature matrices for machine learning. Deep Feature Synthesis (DFS) to perform automated feature engineering. DFS is used to create the “Data Science Machine” to automatically build predictive models for complex, multi-table datasets.<br>
Each table is called an entity in Featuretools. When 2 two entities have a one-to-many relationship, then “one” enitity, is called the “parent entity”. A relationship between a parent and child is defined like this:<br>
(parent_entity, parent_variable, child_entity, child_variable) <br>

A minimal input to DFS is a set of entities, a list of relationships, and the “target_entity” to calculate features for. The ouput of DFS is a feature matrix and the corresponding list of feature defintions.

example:<br>
feature_matrix_customers, features_defs = ft.dfs(entities=entities,
                                                   relationships=relationships,
                                                    target_entity="customers"
<br>
We can change target entity and get feature matrix for any entities of our choice.

In [5]:
import pandas as pd
import urllib
from urllib import request
import featuretools as ft
import numpy as np

url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
data = pd.read_csv(url)
data.head(5)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


### Converting dataset into Entity

In [25]:
entities = {
    "energy" : (data, "id")
}

In [39]:
es=ft.EntitySet("my-entity-set", entities)
es

2018-03-14 19:11:11,554 featuretools.entityset - WARNING    index id not found in dataframe, creating new integer column


Entityset: my-entity-set
  Entities:
    energy (shape = [19735, 30])
  Relationships:
    No relationships

In [45]:
es["energy"].variables

[<Variable: date (dtype: datetime, format: None)>,
 <Variable: Appliances (dtype = numeric, count = 19735)>,
 <Variable: lights (dtype = numeric, count = 19735)>,
 <Variable: T1 (dtype = numeric, count = 19735)>,
 <Variable: RH_1 (dtype = numeric, count = 19735)>,
 <Variable: T2 (dtype = numeric, count = 19735)>,
 <Variable: RH_2 (dtype = numeric, count = 19735)>,
 <Variable: T3 (dtype = numeric, count = 19735)>,
 <Variable: RH_3 (dtype = numeric, count = 19735)>,
 <Variable: T4 (dtype = numeric, count = 19735)>,
 <Variable: RH_4 (dtype = numeric, count = 19735)>,
 <Variable: T5 (dtype = numeric, count = 19735)>,
 <Variable: RH_5 (dtype = numeric, count = 19735)>,
 <Variable: T6 (dtype = numeric, count = 19735)>,
 <Variable: RH_6 (dtype = numeric, count = 19735)>,
 <Variable: T7 (dtype = numeric, count = 19735)>,
 <Variable: RH_7 (dtype = numeric, count = 19735)>,
 <Variable: T8 (dtype = numeric, count = 19735)>,
 <Variable: RH_8 (dtype = numeric, count = 19735)>,
 <Variable: T9 (dtype

In [55]:
es = es.normalize_entity(base_entity_id="energy",
                             new_entity_id="lights",
                             index="Appliances",additional_variables=["T3", "T4", "T5"])

es["lights"].variables

2018-03-14 19:23:18,792 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: energy.Appliances -> lights.Appliances>


[<Variable: Appliances (dtype = index, count = 92)>,
 <Variable: T3 (dtype = numeric, count = 92)>,
 <Variable: T4 (dtype = numeric, count = 92)>,
 <Variable: T5 (dtype = numeric, count = 92)>]

In [56]:
es

Entityset: my-entity-set
  Entities:
    energy (shape = [19735, 24])
    lights (shape = [92, 4])
  Relationships:
    energy.Appliances -> lights.Appliances

In [61]:
es["lights"].head()

,Appliances,T3,T4,T5
Appliances,,,,
10,10,19.700000,18.600000,17.100000
20,20,20.290000,20.533333,18.700000
30,30,20.200000,20.566667,19.200000
40,40,20.230000,20.700000,19.200000
50,50,19.790000,18.926667,17.166667
60,60,19.790000,19.000000,17.166667
70,70,19.790000,18.890000,17.100000
80,80,20.200000,18.963333,17.200000
90,90,20.200000,18.926667,17.166667


In [62]:
es["energy"].head()

,id,date,Appliances,lights,RH_1,RH_2,RH_3,RH_4,RH_5,RH_6,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
id,,,,,,,,,,,,,,,,,,,,,
0,0,2016-01-11 17:00:00,60,30,47.596667,44.790000,44.730000,45.566667,55.200000,84.256667,...,17.033333,45.53,6.600000,733.500000,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,1,2016-01-11 17:10:00,60,30,46.693333,44.722500,44.790000,45.992500,55.200000,84.063333,...,17.066667,45.56,6.483333,733.600000,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2,2016-01-11 17:20:00,50,30,46.300000,44.626667,44.933333,45.890000,55.090000,83.156667,...,17.000000,45.50,6.366667,733.700000,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,3,2016-01-11 17:30:00,50,40,46.066667,44.590000,45.000000,45.723333,55.090000,83.423333,...,17.000000,45.40,6.250000,733.800000,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,4,2016-01-11 17:40:00,60,40,46.333333,44.530000,45.000000,45.530000,55.090000,84.893333,...,17.000000,45.40,6.133333,733.900000,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
5,5,2016-01-11 17:50:00,50,40,46.026667,44.500000,44.933333,45.730000,55.030000,85.766667,...,17.000000,45.29,6.016667,734.000000,92.000000,5.333333,43.833333,4.800000,44.919484,44.919484
6,6,2016-01-11 18:00:00,60,50,45.766667,44.500000,44.900000,45.790000,54.966667,86.090000,...,17.000000,45.29,5.900000,734.100000,92.000000,5.000000,40.000000,4.700000,47.233763,47.233763
7,7,2016-01-11 18:10:00,60,50,45.560000,44.500000,44.900000,45.863333,54.900000,86.423333,...,17.000000,45.29,5.916667,734.166667,91.833333,5.166667,40.000000,4.683333,33.039890,33.039890
8,8,2016-01-11 18:20:00,60,40,45.597500,44.433333,44.790000,45.790000,55.000000,87.226667,...,17.000000,45.29,5.933333,734.233333,91.666667,5.333333,40.000000,4.666667,31.455702,31.455702


In [ ]:
new_relationship = ft.Relationship(es["lights"]["Appliances"],
                                     es["energy"]["Appliances"])

In [63]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                          target_entity="lights")


In [64]:
feature_matrix

,T3,T4,T5,SUM(energy.lights),SUM(energy.RH_1),SUM(energy.RH_2),SUM(energy.RH_3),SUM(energy.RH_4),SUM(energy.RH_5),SUM(energy.RH_6),...,MEAN(energy.rv2),COUNT(energy),NUM_UNIQUE(energy.DAY(date)),NUM_UNIQUE(energy.YEAR(date)),NUM_UNIQUE(energy.MONTH(date)),NUM_UNIQUE(energy.WEEKDAY(date)),MODE(energy.DAY(date)),MODE(energy.YEAR(date)),MODE(energy.MONTH(date)),MODE(energy.WEEKDAY(date))
Appliances,,,,,,,,,,,,,,,,,,,,,
10,19.700000,18.600000,17.100000,10,388.211667,392.656667,377.687500,386.090000,460.230825,829.831111,...,18.903283,9,7,1,2,5,28,2016,1,3
20,20.290000,20.533333,18.700000,170,14075.958929,14260.389544,14008.221917,14003.102333,17767.378927,27685.161240,...,24.866657,343,31,1,5,7,27,2016,1,4
30,20.200000,20.566667,19.200000,780,28573.419167,29275.997506,28520.481087,28059.508548,36518.674745,50411.846572,...,25.960188,723,31,1,5,7,18,2016,1,4
40,20.230000,20.700000,19.200000,2730,81469.864211,82956.507901,81010.238017,80232.707178,103386.057881,144598.758299,...,24.878402,2019,31,1,5,7,19,2016,1,1
50,19.790000,18.926667,17.166667,5210,174458.232753,178660.487326,171553.540294,170013.238402,219820.662122,256046.016930,...,25.104459,4368,31,1,5,7,19,2016,4,1
60,19.790000,19.000000,17.166667,10590,131585.699674,134149.038077,127859.407501,127638.785237,163948.620873,166927.484823,...,25.181673,3282,31,1,5,7,12,2016,4,1
70,19.790000,18.890000,17.100000,7880,62253.329592,62556.575831,60458.791107,60447.136386,79456.418826,77628.319468,...,24.501307,1560,31,1,5,7,25,2016,4,2
80,20.200000,18.963333,17.200000,5460,47813.004756,47304.958316,46461.487159,46449.237613,61777.688307,56761.166294,...,24.656351,1205,31,1,5,7,20,2016,5,2
90,20.200000,18.926667,17.166667,5700,40606.643236,39995.960522,39323.172769,39211.660532,52370.610106,46845.591517,...,25.640040,1015,31,1,5,7,24,2016,3,2
